In [ ]:
import pandas as pd, glob, os
from cassandra.cluster import Cluster
from cassandra.auth import SigV4AuthProvider
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED

REGION   = "us-east-2"                 # change if your Keyspaces region differs
HOST     = f"cassandra.{REGION}.amazonaws.com"
KEYSPACE = "mimic"

auth = SigV4AuthProvider(region_name=REGION)
cluster = Cluster([HOST], port=9142, auth_provider=auth, ssl=True)
session = cluster.connect()
session.execute(f"""
CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
WITH replication = {{'class':'SingleRegionStrategy'}};
""")
session.set_keyspace(KEYSPACE)

# ---------- create tables (raw, NOT aggregated) --------------------------
session.execute("""
CREATE TABLE IF NOT EXISTS drug_by_ethnicity_raw (
    ethnicity text,
    hadm_id   int,
    drug      text,
    dose      double,
    PRIMARY KEY ((ethnicity), drug, hadm_id)
);
""")

session.execute("""
CREATE TABLE IF NOT EXISTS proc_by_age_raw (
    age_group  text,
    icd9_code  text,
    subject_id int,
    proc_name  text,
    PRIMARY KEY ((age_group), icd9_code, subject_id)
);
""")

session.execute("""
CREATE TABLE IF NOT EXISTS icu_los_raw (
    gender     text,
    ethnicity  text,
    icustay_id int,
    los_hours  double,
    PRIMARY KEY ((gender, ethnicity), icustay_id)
);
""")

# ---------- load the CSVs -------------------------------------------------
def bulk_insert(csv_path, table, cols):
    df = pd.read_csv(csv_path)
    prepared = session.prepare(
        f"INSERT INTO {table} ({', '.join(cols)}) VALUES ({', '.join(['?']*len(cols))})"
    )
    for row in df.itertuples(index=False):
        session.execute(prepared, row)

bulk_insert("/home/ec2-user/data/q1_drug_ethnicity.csv",
            "drug_by_ethnicity_raw",
            ["ethnicity","hadm_id","drug","dose"])

bulk_insert("/home/ec2-user/data/q2_proc_age.csv",
            "proc_by_age_raw",
            ["age_group","icd9_code","subject_id","proc_name"])

bulk_insert("/home/ec2-user/data/q3_icu_los.csv",
            "icu_los_raw",
            ["gender","ethnicity","icustay_id","los_hours"])

print("✓ All three tables populated.")


In [ ]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import SigV4AuthProvider

REGION="us-east-2"
session = Cluster([f"cassandra.{REGION}.amazonaws.com"], port=9142,
                  auth_provider=SigV4AuthProvider(region_name=REGION),
                  ssl=True).connect("mimic")

# ---------- Question 1 example (top drug by ethnicity) -------------------
rows = session.execute("SELECT * FROM drug_by_ethnicity_raw WHERE ethnicity=%s", ("WHITE",))
df   = pd.DataFrame(rows, columns=["ethnicity","hadm_id","drug","dose"])
top  = (df.groupby("drug")["dose"].sum()
          .nlargest(1)
          .reset_index())
print(top)

# ---------- Question 2 example (top 3 procedures per age group) ----------
age_grp = "20-49"
rows = session.execute("SELECT * FROM proc_by_age_raw WHERE age_group=%s", (age_grp,))
df = pd.DataFrame(rows, columns=["age_group","icd9_code","subject_id","proc_name"])
top3 = (df.groupby(["icd9_code","proc_name"])
          .size().sort_values(ascending=False).head(3))
print(top3)

# ---------- Question 3 example (ICU-LOS distribution by gender) ----------
rows = session.execute("SELECT * FROM icu_los_raw WHERE gender=%s AND ethnicity=%s",
                       ("M", "WHITE"))
df = pd.DataFrame(rows, columns=["gender","ethnicity","icustay_id","los_hours"])
print(df.los_hours.describe())
